In [2]:
import pandas as pd
from google.colab import drive
import matplotlib.pyplot as plt
import numpy as np

In [3]:
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [4]:
df=pd.read_csv('/content/gdrive/MyDrive/fraudTest.csv')

In [5]:
df.columns

Index(['Unnamed: 0', 'trans_date_trans_time', 'cc_num', 'merchant', 'category',
       'amt', 'first', 'last', 'gender', 'street', 'city', 'state', 'zip',
       'lat', 'long', 'city_pop', 'job', 'dob', 'trans_num', 'unix_time',
       'merch_lat', 'merch_long', 'is_fraud'],
      dtype='object')

# 2.) Select four columns to use as features (one just be trans_date_trans)

In [6]:
df_select = df[['trans_date_trans_time','category','amt','city_pop',"is_fraud"]].copy()

In [7]:
df_select['trans_date_trans_time'] = pd.to_datetime(df_select['trans_date_trans_time'])

In [8]:
df_select.columns

Index(['trans_date_trans_time', 'category', 'amt', 'city_pop', 'is_fraud'], dtype='object')

# 3.) Create a unique variable out of trans_date.

In [9]:
df_select['time_var']=[i.minute for i in df_select['trans_date_trans_time']]

In [10]:
X = pd.get_dummies(df_select, columns=["category"]).drop(['trans_date_trans_time','is_fraud'],axis = 1)
y = df["is_fraud"]

In [11]:
X.head()

,amt,city_pop,time_var,category_entertainment,category_food_dining,category_gas_transport,category_grocery_net,category_grocery_pos,category_health_fitness,category_home,category_kids_pets,category_misc_net,category_misc_pos,category_personal_care,category_shopping_net,category_shopping_pos,category_travel
0,2.86,333497,14,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,29.84,302,14,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,41.28,34496,14,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,60.05,54767,15,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,3.19,1126,15,0,0,0,0,0,0,0,0,0,0,0,0,0,1


# 4.) Oversample the data (this will be your training data).

In [13]:
resample_X = X
resample_y = y

# 5.) Train a Logistic regression.

In [14]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_normalized = scaler.fit_transform(resample_X)

In [15]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression().fit(X_normalized, resample_y)

# 6.) The company you are working for wants to target at a False Positive rate of 5% what threshold should you use? (Use oversampled data)

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [17]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

# Assume you have already trained your logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Get the confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Define the desired false negative rate
target_fnr = 0.05

# Get the total number of negative instances
num_negatives = conf_matrix[0][0] + conf_matrix[0][1]

# Get the total number of false negatives
num_false_negatives = conf_matrix[1][0]

# Calculate the threshold at which the false negative rate is less than or equal to the target
threshold = num_false_negatives / num_negatives

# Print the threshold
print(threshold)

0.0038476128542784373


# 7.) If the company makes .02*amt on True transactions and loses -amt on False (Use original data)

In [18]:
df_temp = df_select.copy()
df_temp["pred"] = log_reg.predict(resample_X)
df_temp = df_temp[["pred", "is_fraud", "amt"]]


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


In [19]:
df_temp.head()

,pred,is_fraud,amt
0,0,0,2.86
1,0,0,29.84
2,0,0,41.28
3,0,0,60.05
4,0,0,3.19


In [20]:
df_temp['factor'] = np.where(df_temp['pred'] == df_temp['is_fraud'], 0.02, -1) 
df_temp['ind_profit'] = df_temp['amt']*df_temp['factor']
Profit = sum(df_temp['ind_profit']) 
Profit

-5341377.2929999605

# 8.) Using Logistic Regression Lasso to inform you. Would you use the selected features in a trusted prediction model?

In [29]:
LogLasso = LogisticRegression(penalty='l1', solver ='liblinear') 
LogLasso.fit(X_train, y_train)


LogisticRegression(penalty='l1', solver='liblinear')

In [33]:
arr=LogLasso.coef_
arr=arr.ravel()

In [34]:
float_arr = [float(x) for x in arr]
float_arr


[0.002112856001471841,
 -3.965469696054225e-07,
 -0.0024081247021763646,
 -4.155869144912285,
 -4.122798819009337,
 -3.514814972644669,
 -3.6836020310970294,
 -2.3745324786372577,
 -4.042713025773719,
 -4.249275276793081,
 -4.1251591562284,
 -2.412276684632321,
 -3.8909947348416747,
 -3.824148754997276,
 -2.3516635775984054,
 -3.569767751164847,
 -8.48061622968322]

Most features seem significant